## Библиотеки и настройки ##

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math
import os

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium.webdriver import Chrome

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/' + input() + '/'
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси проще прописать вручную, ввод со строки постоянно глючит

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

#proxy_number = 0

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
chrome_driver_path = 'C:/Users/bgi/Anaconda3/chromedriver/chromedriver'

In [ ]:
# запускаю драйвер, проверяю настройки

driver = Chrome('C:/Users/bgi/Anaconda3/chromedriver/chromedriver', options=opt)

#driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

In [ ]:
driver.find_element(By.LINK_TEXT, 'Последняя').text

## Собираю ссылки ##

In [ ]:
# ссылка

# Кошки
url = 'https://poiskzoo.ru/propala-koshka/page-1' # пропала
#url = 'https://poiskzoo.ru/najdena-koshka/page-1' # найдена

# Собаки
#url = 'https://poiskzoo.ru/propala-sobaka/page-1'
#url = 'https://poiskzoo.ru/najdena-sobaka/page-1'

# Остальные
#url = 'https://poiskzoo.ru/najdena-poteryashka/page-1'
#url = 'https://poiskzoo.ru/propala-poteryashka/page-1'

### Не забудь зайти на посмотреть количество страниц с объявлениями и указать их в коде ###

In [ ]:
#Надо проверить открылась ли страница, и если просит, ввести капчу

driver.get(url)

In [ ]:
# обрезаю ссылки

page_url = url.replace('page-1', 'page-')
ads_category = page_url.replace('https://poiskzoo.ru/', '').replace('/page-', '')

In [ ]:
%%time

# собираю ссылки на объявления

ads_id = []


# количество страниц нужно указать вручную
for p in tqdm_notebook(range(1, 372)): # пока вручную, но, вообще, можно автоматизировать
    sleep(3)
    url = page_url +str(p)
        
    driver.get(url)
    text = str(driver.page_source)
    
    # регулярками вытаскиваю ссылки
    need_text = re.findall(r'a href="/\w+/' +ads_category +'/\d+', text)

    
    # удаляю дубликаты
    need_text = list(set(need_text))
    
    for i in need_text: # удаляю лишнее, теперь ссылка будет /город/najdena-зверюшка/хххх - номер
        ads_id += re.findall(r'/\w+/' +ads_category +'/\d+', i)     
    

In [ ]:
# для простоты, сразу завожу список с рабочими ссылками

links = []

for id in ads_id:
    links += ['https://poiskzoo.ru' +id]

In [ ]:
links[:5]

In [ ]:
len(links)

In [ ]:
ads_list = pd.DataFrame(links)

In [ ]:
ads_list.to_excel('C:/00_Projects/Cats/ads_list_4.xlsx', encoding='utf8', index=False)

In [ ]:
%%time


# список для данных
data_ads = []

# код парсера

for ads_url in tqdm_notebook(links[:101]): # цикл по списку ссылок
    driver.get(ads_url)
    sleep(2)
    
    
    try: # заголовок объявления
        ads_topic = driver.find_element_by_tag_name('h1[itemprop="name"]').text
        
    except:
        ads_topic = 'нет'
        
    
    try: # дата объявления
        ads_date = driver.find_element_by_class_name('bd_item_details_full .bd_item_date').text
        
    except:
        ads_date = 'нет'
                
    
    try: # город объявления
        ads_city = driver.find_element_by_class_name('bd_item_city[itemprop="addressLocality"]').find_element_by_tag_name('a').text
    
    except:
        ads_city = 'нет'
        
        
    try: # 
        ads_text = driver.find_element_by_id('raspechat').text
        
    except:
        ads_text = 'нет'
    


    data_ads.append([ads_date, ads_city, ads_topic, ads_text, ads_url])

In [ ]:
# преобразовываем в датафрейм

head = ['date', 'city', 'topic', 'text', 'url']
df = pd.DataFrame(data_ads, columns=head)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# записываю в файл

df.to_excel('C:/00_Projects/Cats/dogs2/1.xlsx', encoding='utf8', index=False)

In [ ]:
driver.close()